# Temporal Difference Learning for 3D Tic Tac Toe

This notebook contains the implementation of a Temporal Difference (TD) learning model using a Deep Q-Network (DQN) for playing 3D 4x4x4 Tic Tac Toe. The implementation is based on the approach outlined in the provided paper.


In [13]:
# Import necessary libraries
import numpy as np
import os
import random

# Setting Directory
# os.chdir('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/')
os.chdir('C:/Users/HP/Documents/GitHub/reinforcement_learning/Project/')

from python_scripts import state_formulation, utils, algorithm, tictactoe_4x4
import torch
import torch.nn as nn
import torch.optim as optim
!pip install torchinfo
from torchinfo import summary
import torch.nn.init as init
from tqdm.autonotebook import tqdm

In [14]:
class customDotProduct(nn.Module):
    def __init__(self, input_size, output_size, block_size = 4):
        super(customDotProduct, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.block_size = block_size
        # Convert structure_weight to nn.Parameter
        self.structure_weight = torch.zeros((self.output_size, self.input_size))
        self.structure_weight = self.get_block_weights(self.structure_weight, block_size)
        self.structure_weight = nn.ParameterList([nn.Parameter(sw.float()) for sw in self.structure_weight])

    def get_block_weights(self, weight_list, block_size):
        for i in range(0, 304, block_size):
            weight_list[i: i + block_size, i: i + block_size] = init.xavier_normal_(torch.randn(block_size, block_size))
        learnable_blocks = [weight_list[i:i + block_size, i:i + block_size] for i in range(0, weight_list.shape[0], block_size)]
        updated = [block for block in learnable_blocks]
        return updated
    
    def forward(self, feature_map):
        self.feature_map = [fm.float() for fm in feature_map]
        # Calculate dot products and concatenate along dim = 1
        concatenated_products = torch.cat([torch.matmul(fm.unsqueeze(0), sw) for fm, sw in zip(self.feature_map, self.structure_weight)], dim = 1)
        return concatenated_products

In [15]:
# # Testing Code
# weights = torch.zeros((304, 304))

# block_size = 4
# for i in range(0, 304, block_size):
#     weights[i: i + block_size, i: i + block_size] = torch.ones(block_size, block_size)
# learnable_blocks = [weights[i:i + block_size, i:i + block_size] for i in range(0, weights.shape[0], block_size)]
# weights = [init.xavier_normal_(block) for block in learnable_blocks]

# print(f'Before Update: Weights = {weights[0]} \n')

# # Assume some loss function and optimizer have been defined
# custom_dot_product_module = customDotProduct(304, 304)
# loss_function = nn.MSELoss()
# optimizer = torch.optim.SGD(custom_dot_product_module.parameters(), lr = 0.01)

# # Example training loop iteration
# optimizer.zero_grad()  # Clear gradients
# output = custom_dot_product_module(rows)  # Perform forward pass
# loss = loss_function(output, torch.randn(1, 304))  # Compute loss
# loss.backward()  # Perform backward pass
# optimizer.step()  # Update weights
# print(f'After Update: Weights = {custom_dot_product_module.structure_weight[0]}')


In [16]:
class StructuredLinear(nn.Module):
    def __init__(self):
        super(StructuredLinear, self).__init__()

    def get_rows(self, input_tensor):
        # Get diagonals (across 2 faces),digonals (across 3 faces) and horizontal and vertical rows
        diag_two_faces = []
        diag_two_faces.extend(
            [torch.diagonal(input_tensor[i, :, :]), torch.diagonal(input_tensor[:, i, :]), torch.diagonal(input_tensor[:, :, i]), 
            torch.diagonal(torch.fliplr(input_tensor)[i, :, :]), torch.diagonal(torch.fliplr(input_tensor)[:, i, :]), torch.diagonal(torch.fliplr(input_tensor)[:, :, i])] 
            for i in range(input_tensor.shape[0]))
        diag_two_faces = [item for sublist in diag_two_faces for item in sublist]
        
        diag_three_faces = []
        diag_three_faces = [[[[input_tensor[i, i, i], input_tensor[3 - i, i, i], input_tensor[i, 3 - i, i], input_tensor[i, i, 3 - i]] 
                            for i in range(4)][k][j] for j in range(4) for k in range(4)][l:l + 4] for l in range(0, 16, 4)]
        diag_three_faces = [torch.tensor([t.item() for t in row]) for row in diag_three_faces]

        horizontal_and_vertical_rows = []
        horizontal_and_vertical_rows.extend([input_tensor[i, j, :], input_tensor[i, :, j], input_tensor[:, i, j]]
                                            for i in range(input_tensor.shape[0]) for j in range(input_tensor.shape[0]))
        horizontal_and_vertical_rows = [item for sublist in horizontal_and_vertical_rows for item in sublist]
        
        return horizontal_and_vertical_rows + diag_two_faces + diag_three_faces

    def forward(self, x):
        rows = self.get_rows(x)
        return rows

class MyNeuralNetwork(nn.Module, tictactoe_4x4.TicTacToe4x4x4):
    def __init__(self, num_detectors):
        super(MyNeuralNetwork, self).__init__()

        self.num_detectors = num_detectors
        self.structured_layer = StructuredLinear()
        
        self.custom_operation_layer = customDotProduct(input_size = num_detectors * 4, output_size = num_detectors * 4)

        self.second_layer = nn.Linear(num_detectors * 4, 32, bias = False)
        init.xavier_normal_(self.second_layer.weight)

        self.output_layer = nn.Linear(32, 1, bias = False)
        self.act = nn.Tanh()

    def forward(self, x):
        x = self.structured_layer(x)
        x = self.custom_operation_layer(x)
        x = self.act(x) # --> Tanh
        x = self.second_layer(x)
        x = self.act(x)
        x = self.output_layer(x)
        return x
    
# Example usage
model = MyNeuralNetwork()


In [17]:
summary(model, input_size = [4, 4, 4], col_names = ['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MyNeuralNetwork                          [4, 4, 4]                 [1, 1]                    --
├─StructuredLinear: 1-1                  [4, 4, 4]                 [4]                       --
├─customDotProduct: 1-2                  [4]                       [1, 304]                  1,216
├─Tanh: 1-3                              [1, 304]                  [1, 304]                  --
├─Linear: 1-4                            [1, 304]                  [1, 32]                   9,728
├─Tanh: 1-5                              [1, 32]                   [1, 32]                   --
├─Linear: 1-6                            [1, 32]                   [1, 1]                    32
Total params: 10,976
Trainable params: 10,976
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0

In [18]:
model = MyNeuralNetwork()

In [19]:
# Training parameters
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_function = nn.MSELoss()
EPSILON = 0.1
GAMMA = 0.9
reward = torch.randint(low = 0, high = 2, size = (4, 4, 4))

In [20]:
def get_next(current_state):
    action_space = torch.stack(torch.where(current_state == 0), -1)
    afterstates = []
    for i in action_space:
        afterstates.append(current_state.detach().clone())
        afterstates[-1][tuple(i)] = 1
    return zip(action_space, torch.stack(afterstates))

def e_greedy(current_state, temp_dict):
    if np.random.random() > EPSILON:
        return max(temp_dict, key = lambda k: temp_dict[k])
    action_space = torch.stack(torch.where(current_state == 0), -1)
    index = torch.randint(low = 0, high = len(action_space), size = (1,))
    return action_space[index]

def train_td_model(model, num_episodes):
    for episode in tqdm(range(num_episodes)):
        current_state = torch.randint(low = -1, high = 2, size = (4, 4, 4))
        prev_afterstate = None
        for t in range(1000):
            if 0 not in current_state:
                break
            temp_dict = {}
            for a, afterstate in get_next(current_state):
                temp_dict[a] = model.forward(afterstate)
            action = e_greedy(current_state, temp_dict)

            current_state[action] = 1
            if t != 0:
                v_new = reward[prev_afterstate] + (GAMMA * model.forward(current_state))
                v = model.forward(prev_afterstate)
                loss = loss_function(v, v_new)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            prev_afterstate = current_state
            if 0 not in current_state:
                break
            current_state[torch.where(current_state == 0)] = -1

In [21]:
# Example usage
num_episodes = 1000  # Number of episodes for training
train_td_model(model, num_episodes)  # Train the model

# Save the trained model
# os.makedirs('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models', exist_ok = True)
# model_path = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models/td_tictactoe_model.pth'
# save_model(model, model_path)

# model_path

  0%|          | 0/1000 [00:00<?, ?it/s]